<a href="https://colab.research.google.com/github/DDiekmann/Applied-Verification-Lab-Neural-Networks/blob/main/Tutorial/Eran2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
%%bash

git clone https://github.com/eth-sri/ERAN.git

sudo apt install m4
sudo apt install build-essential
sudo apt install autoconf
sudo apt install libtool
sudo apt install texlive-latex-base

cd ERAN
sudo bash ./install.sh
pip install -r requirements.txt

In [ ]:
%pip uninstall -y pillow
%pip install pillow

# Currently you have to do this and restart the runtime
# This needs to go by removing pillow from the requirements.txt before installing it or installing the packages manually

In [ ]:
%%capture

%pip install onnx onnxruntime

import torch
from torch import nn
import numpy as np
import torch.onnx
import sys

import matplotlib.pyplot as plt

!wget --no-cache --backups=1 {'https://raw.githubusercontent.com/DDiekmann/Applied-Verification-Lab-Neural-Networks/main/lib/iris_trainer.py'}
import iris_trainer

!wget --no-cache --backups=1 {'https://raw.githubusercontent.com/DDiekmann/Applied-Verification-Lab-Neural-Networks/main/lib/plots.py'}
import plots

!wget --no-cache --backups=1 {'https://raw.githubusercontent.com/DDiekmann/zonotpy/main/src/zonotope.py'}
from zonotope import zono

!wget --no-cache --backups=1 {'https://raw.githubusercontent.com/DDiekmann/zonotpy/main/src/interval_abstraction.py'}
import interval_abstraction as ia

!wget --no-cache --backups=1 {'https://raw.githubusercontent.com/DDiekmann/zonotpy/main/src/nn_functions.py'}
import nn_functions as zf

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim, number_of_neurons):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_dim, number_of_neurons),
            nn.ReLU(),
            nn.Linear(number_of_neurons, output_dim),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

names, feature_names, X, y, X_scaled, X_train, X_test, y_train, y_test = iris_trainer.load_dataset()

model = iris_trainer.train_model(
    NeuralNetwork(input_dim=X_train.shape[1], output_dim=3, number_of_neurons=10), 
    epochs=100, 
    X_train=X_train, 
    X_test=X_test, 
    y_train=y_train, 
    y_test=y_test)

plots.show_plots(names, feature_names, X, y, title = 'Ground Truth')
plots.show_plots(names, feature_names, X, iris_trainer.predict(X_scaled, model), title = 'Classification from our network')

In [ ]:
print(X[0])
print(y[0])

In [ ]:
model_parameters = [param.cpu().detach().numpy() for param in model.parameters()]
weights = model_parameters[0::2]
print(weights)
biases = model_parameters[1::2]
print(biases)

In [ ]:
model_filename = "iris_net.onnx"
dummy_input=torch.randn(1, 4)

# set model to eval mode
model.eval()

# create a dummy input in the shape of the input values
device = "cuda" if torch.cuda.is_available() else "cpu"
dummy_input = dummy_input.to(device)

torch.onnx.export(model,
                  dummy_input,
                  model_filename,
                  export_params=True,
                  verbose=False,
                  input_names=['data'],
                  output_names=['classification'],
                  )

In [ ]:
%%writefile input_box.txt
[5.1, 5.11]
[3.5, 3.51]
[1.4, 1.41]
[0.2, 0.21]

In [ ]:
%%writefile zonotype_example.txt
4
3
5.1 0.03 0.0
3.5 0.0 0.03
1.4 0.03 0.0 
0.2 0.0 0.03

In [ ]:
%%bash
cd ERAN/tf_verify/

python3 . --netname ../../iris_net.onnx --zonotope ../../zonotype_example.txt --input_box ../../input_box.txt --domain deepzono --debug true 

In [ ]:
def relu(x):
  if x > 0: return x
  return 0

input_nodes = [5.1, 3.5, 1.4, 0.2]

def calculate_point(input_node):
  hidden_nodes = []
  for i in range(10):
    z = 0
    for w in range(4):
      z = z + (input_nodes[w] * weights[0][i][w])
    hidden_nodes.append(z)

  for i in range(10):
    hidden_nodes[i] = hidden_nodes[i] + biases[0][i]


  for i in range(10):
    hidden_nodes[i] = relu(hidden_nodes[i])


  output_nodes = []
  for i in range(3):
    z = 0
    for w in range(10):
      z = z + (hidden_nodes[w] * weights[1][i][w])
    output_nodes.append(z)

  for i in range(3):
    output_nodes[i] = output_nodes[i] + biases[1][i]

  return output_nodes

In [ ]:
def create_plot():
  fig, ax = plt.subplots(2, 2)

  fig.set_size_inches(18, 14)

  ax[0][0].set_title("Input")
  ax[0][0].set_xlim(4, 8)
  ax[0][0].set_ylim(1.5, 4.7)
  ax[0][1].set_title("Output")
  ax[0][1].set_xlim(4, 8)
  ax[0][1].set_ylim(1.5, 4.7)

  ax[1][0].set_title("Input")
  ax[1][0].set_xlim(0.8, 7.1)
  ax[1][0].set_ylim(-1.2, 3.8)
  ax[1][1].set_title("Output")
  ax[1][1].set_xlim(0.8, 7.1)
  ax[1][1].set_ylim(-1.2, 3.8)
  return fig, ax

fig, ax = create_plot()

input = zono(values = np.array([[5.1, 0.1, 0], [3.5, 0, 0.1], [1.4, 0.1, 0], [0.2, 0, 0.1]]))
print(input.to_intervals())
k1, k2 = input.split()
i1, i2 = k1.split()
i3, i4 = k2.split()

input_nodes = [i1, i2, i3, i4]

k1.visualize(shape=True, fig=fig, ax=ax[0][0])
k2.visualize(shape=True, fig=fig, ax=ax[1][0])

hidden_nodes = []
for i in range(10):
  z = zono()
  for w in range(4):
    z = z + (input_nodes[w] * weights[0][i][w])
  hidden_nodes.append(z)

for i in range(10):
  hidden_nodes[i] = hidden_nodes[i] + zono(values = np.array([[biases[0][i]]]))


for i in range(10):
  hidden_nodes[i] = zf.relu(hidden_nodes[i])


output_nodes = []
for i in range(3):
  z = zono()
  for w in range(10):
    z = z + (hidden_nodes[w] * weights[1][i][w])
  output_nodes.append(z)

for i in range(3):
  output_nodes[i] = output_nodes[i] + zono(values = np.array([[biases[1][i]]]))

output = output_nodes[0].combine(output_nodes[1]).combine(output_nodes[2])

print(output.to_intervals())
print(output)

print(output.upper_bound(1))
print(output.lower_bound(1))

print(output.upper_bound(2))
print(output.lower_bound(2))

print(output.upper_bound(3))
print(output.lower_bound(3))

plt.show()